In [6]:
import deepemogp.feature_extractor as feature_extractor
import deepemogp.signal.physio as physio
import deepemogp.signal.face as face
import deepemogp.signal.annotation as annotation
import deepemogp.datasets as datasets

# from: https://github.com/SheffieldML/GPy
#import GPy
# from: https://github.com/SheffieldML/PyDeepGP
#import deepgp

import pandas as pd
import numpy as np


In [7]:
show = True
loadData = True
learnModel = True

In [3]:

#if loadData:

# definition of the feature extractors to be used later
f1 = feature_extractor.FE('wavelet')
f2 = feature_extractor.FE('wavelet')
f3 = feature_extractor.FE('mean')

In [4]:

# definition of the physiological signals to be extracted
skt = physio.SKT(f1)
hr  = physio.HR(f2)
#of  = face.FP_OF(f3)

# definition of the emotional annotation to be extracted
#va = annotation.VA('valence', f3)
#ar = annotation.VA('arousal', f3)

In [5]:
# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr}, subjects={'1_1'})
print(d)

>> Loading HR for subject 1 and session 1 from dataset FEAR
loaded npy file
[[-1.23251646e-02 -1.24299099e-03  6.87468359e-03 -1.33069141e-02
   3.77871677e-03  5.38126340e-03]
 [-7.14022533e-03  2.38219983e-02 -2.40823339e-02 -1.33517513e-02
   3.33270339e-02 -3.71429790e-02]
 [-1.30361924e-02  2.73208844e-02 -1.55380287e-02  2.24158020e-02
  -2.87222212e-02  3.32277981e-02]
 [ 7.73560820e-03 -2.28481948e-02  1.97787817e-02 -7.99560800e-05
   3.75516200e-03  2.81653719e-03]
 [ 3.84237696e-02 -3.02251992e-02  6.20302140e-03 -3.18275300e-04
  -1.41463951e-02  1.21992647e-02]
 [ 1.47366489e-02 -2.37793141e-02  6.99885561e-03 -1.40414152e-02
   2.99704046e-02 -2.75244009e-02]
 [-2.94778244e-02  2.09808506e-02 -1.53140144e-02  2.23437462e-02
  -2.43637085e-02  1.57482863e-02]
 [ 2.46835371e-02 -6.96623928e-03 -1.61187939e-02  1.35833851e-02
  -1.20834169e-02 -2.57044398e-02]
 [ 1.21694732e-02  2.35777340e-03 -3.48878605e-03 -8.83453566e-03
   1.72934473e-02  2.30516123e-03]
 [ 3.52726101e-

TypeError: 'NoneType' object is not iterable

In [ ]:
data = np.load('../fear_gen/prova.npy', allow_pickle=True).item()
eda = data['eda']

In [ ]:
signal_ = eda
signal_.feature_ext.extract_feat(signal_, show=show)

In [ ]:
for s in d.signals:
	# preprocess ...
	s.preprocess(show=show)

	# ... and extract features from each signal type
	s.feature_ext.extract_feat(s, show=show)

	print(s)

for a in d.annotations:
	# preprocess ...
	a.preprocess(ewe=False, show=show)

	# ... and extract features for each annotation type
	a.feature_ext.extract_feat(a, show=show)
	print(a)


# Saving a model
# np.save('data.npy', d)

'''
else:

	d = np.load('data.npy') # Load the data
	d = d.item()
'''

In [ ]:

# create model
selected_view = 'HR'

# observed inputs variables (annotations)
X = pd.concat([a.features for a in d.annotations], axis=1).values
# observed output variables (measured signals)
Y = [s.features.values for s in d.signals if s.name==selected_view][0]

Xmean = X.mean()
Xstd = X.std()

#X-=Xmean
#X/=Xstd

N = len(X)

# Number of latent dimensions (single hidden layer, since the top layer is observed)
Q = [Y.shape[1], 6, X.shape[1]]

k1 = GPy.kern.RBF(Q[1], ARD=True) + GPy.kern.Bias(Q[1]) + GPy.kern.White(Q[1])

k2 = GPy.kern.Linear(Q[2], ARD=False)

# Dimensions of the MLP back-constraint if set to true
encoder_dims=[[300],[150]]

#inits = ['PPCA', 'PPCA' , X]

# deepGP
#m = deepgp.DeepGP([d1, Q1, X.shape[1]], Y=Ytr, X=Xtr,
m = deepgp.DeepGP(Q, Y=Y, X=X,
	kernels=[k1, k2],
	num_inducing=N,
	back_constraint=True,
	encoder_dims=encoder_dims)


if learnModel:
	#--------- Optimization ----------#
	for i in range(len(m.layers)):
		# Make sure initial noise variance gives a reasonable signal to noise ratio.
		# Fix to that value for a few iterations to avoid early local minima
		output_var = m.layers[i].Y.var() if i==0 else m.layers[i].Y.mean.var()
		m.layers[i].Gaussian_noise.variance = output_var *  0.1
		m.layers[i].Gaussian_noise.variance.fix()

	m.optimize(max_iters=1000, messages=1)
	deepgp.util.check_snr(m)

	# Now unfix noise and learn normally.
	for i in range(len(m.layers)):
		m.layers[i].Gaussian_noise.variance.unfix()

	#m.optimize_restarts(parallel=False, messages=1, robust=True, num_restarts=3, max_iters=250)
	m.optimize(max_iters=10000, messages=1)
	deepgp.util.check_snr(m)

	print(m)

	# Saving a model
	np.save('model_save.npy', m.param_array)

else:

	m.update_model(False) # do not call the underlying expensive algebra on load
	m.initialize_parameter() # Initialize the parameters (connect the parameters up)
	m[:] = np.load('model_save.npy') # Load the parameters
	m.update_model(True) # Call the algebra only once


# Show ARD results
import matplotlib.pyplot as plt
m.obslayer.kern.plot_ARD('rbf')
plt.show()


# Generate session - Experiment 1
from deepemogp.signal.utils import utils
utils.generate_session(m, [selected_view], d)









# pred = m.predict(Xts)[0]

# rmse = np.sqrt(((pred.flatten() - Yts.flatten()) ** 2).mean())

# print "prediction RMSE: %f" % (rmse)


# import matplotlib.pyplot as plt
# f = plt.figure(); plt.plot(pred.T, 'x-');  plt.plot(Yts.T, 'o'); f.show();